In [13]:
import os

# so I don't 
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="-1"  #prevent access to GPU for inference

import pandas as pd
pd.options.display.max_rows = 150
pd.options.display.max_colwidth = 500
import json
from mdparse import transform_pre_rules, compose
from pathlib import Path
from tqdm import tqdm_notebook

from fastai.text import TextLMDataBunch as lmdb
from fastai.text.transform import Tokenizer
import pandas as pd
from pathlib import Path
from random import shuffle

from fastai.text.models import AWD_LSTM, awd_lstm_lm_config
from fastai.text import TextLMDataBunch as lmdb, load_data
from fastai.text.learner import language_model_learner
from fastai.basic_train import load_learner
from fastai.callbacks import EarlyStoppingCallback, SaveModelCallback, ReduceLROnPlateauCallback, CSVLogger

In [14]:
awd_lstm_lm_config

{'emb_sz': 400,
 'n_hid': 1150,
 'n_layers': 3,
 'pad_token': 1,
 'qrnn': False,
 'bidir': False,
 'output_p': 0.1,
 'hidden_p': 0.15,
 'input_p': 0.25,
 'embed_p': 0.02,
 'weight_p': 0.2,
 'tie_weights': True,
 'out_bias': True}

In [109]:
path = Path('lang_model_debug/')

def pass_through(x):
    return x


valid_df = pd.read_pickle('valid_df.pkl').dropna().drop_duplicates().head(25)
train_df = valid_df

In [110]:
train_df.shape

(25, 2)

In [111]:
sample_train_df = train_df.sample(frac=.01)

In [112]:
tokenizer = Tokenizer(pre_rules=[pass_through], n_cpus=30)

data_lm = lmdb.from_df(path=path,
                       train_df=train_df,
                       valid_df=valid_df,
                       text_cols='text',
                       tokenizer=tokenizer)

In [113]:
learn = language_model_learner(data=data_lm,
                               arch=AWD_LSTM,
                               model_dir=Path('models_' + tst.id),
                               pretrained=False)

In [114]:
learn.path = Path('hp_tune/lang_model_debug_v2')

In [115]:
learn.model_dir

PosixPath('models_c0xgyzey')

In [129]:
!ls lang_model/

data_save.pkl  history.csv  models


In [116]:
escb = EarlyStoppingCallback(learn=learn, patience=4)
smcb = SaveModelCallback(learn=learn)
rpcb = ReduceLROnPlateauCallback(learn=learn, patience=3)
csvcb = CSVLogger(learn=learn)
callbacks = [escb, smcb, rpcb, csvcb, wb]

In [117]:
learn.fit(1, callbacks=callbacks)

epoch,train_loss,valid_loss,accuracy,time
0,6.150044,5.994513,0.127567,00:42


wandb: Wandb version 0.8.1 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Better model found at epoch 0 with valid_loss value: 5.994512557983398.


In [130]:
learn.wd

0.01

### Make Wandb Callback

In [40]:
from fastai.callback import Callback
import wandb

In [76]:
tst = wandb.init()

W&B Run: https://app.wandb.ai/github/issues_lang_model/runs/c0xgyzey
Call `%%wandb` in the cell containing your training loop to display live results.


In [118]:
tst.name

'c0xgyzey'

In [124]:
learn.path

PosixPath('hp_tune/lang_model_debug_v2')

In [87]:
tst.id

'c0xgyzey'

In [122]:
learn.path/learn.model_dir

PosixPath('hp_tune/lang_model_debug_v2/models_c0xgyzey')

In [126]:
!ls hp_tune/lang_model_debug_v2

history.csv  models_c0xgyzey


In [127]:
!ls lang_model_debug_v2

history.csv  models  models_c0xgyzey


In [98]:
learn.save(tst.id)

In [43]:
wandb.save(str(learn.path))

wandb: Wandb version 0.8.1 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


['./wandb/run-20190529_195403-y1j0piga/lang_model_debug']

In [72]:
class wandbCallback(Callback):
    def __init__(self, Learner):
        self.learn=Learner
    
    def on_epoch_end(self, **kwargs):
        train = kwargs['train']
        loss = kwargs['smooth_loss']
        epoch = kwargs['epoch']
        val_loss, val_metrics = kwargs['last_metrics']
        
        if train:
            wandb.log({'loss': loss, 'epoch':epoch})
            
        else:
            wandb.log({'val_loss': val_loss, 'val_accuracy':val_metrics})
        
    def on_step_end(self, **kwargs):
        loss = kwargs['last_loss']
        step_num = kwargs['iteration']
        
        wandb.log({'loss':loss, 'step':step_num})
        
wb = wandbCallback(learn)

In [16]:
type(learn.callbacks[0])

fastai.callbacks.rnn.RNNTrainer

In [18]:
! ls lang_model_debug/

history.csv  models


In [20]:
data_lm.save()

In [75]:
wandb.id

AttributeError: module 'wandb' has no attribute 'id'

## Reload

In [41]:
awd_lstm_lm_config

{'emb_sz': 800,
 'n_hid': 1800,
 'n_layers': 3,
 'pad_token': 1,
 'qrnn': False,
 'bidir': False,
 'output_p': 0.1,
 'hidden_p': 0.15,
 'input_p': 0.25,
 'embed_p': 0.02,
 'weight_p': 0.2,
 'tie_weights': True,
 'out_bias': True}

In [62]:
data_lm = load_data(path, bs=128)

In [19]:
learn = language_model_learner(data=data_lm,
                               arch=AWD_LSTM,
                               pretrained=False,
                               config=awd_lstm_lm_config.update(dict(n_hid=1800, emb_sz=800)),
                              )

#learn.load('bestmodel')

In [20]:
learn.lr_find(num_it=1)

LR Finder is complete, type {learner_name}.recorder.plot() to see the graph.


In [21]:
learn.path

PosixPath('lang_model_debug')

In [31]:
learn.summary()

Layer (type)         Output Shape         Param #    Trainable 
RNNDropout           [70, 800]            0          False     
______________________________________________________________________
RNNDropout           [70, 1800]           0          False     
______________________________________________________________________
RNNDropout           [70, 1800]           0          False     
______________________________________________________________________
Linear               [70, 1036]           829,836    True      
______________________________________________________________________
RNNDropout           [70, 800]            0          False     
______________________________________________________________________

Total params: 829,836
Total trainable params: 829,836
Total non-trainable params: 0

In [5]:
data_lm = load_data(path, bs=128)


learn = language_model_learner(data=data_lm,
                               arch=AWD_LSTM,
                               pretrained=False)

learn.load('bestmodel')
# callbacks
escb = EarlyStoppingCallback(learn=learn, patience=4)
smcb = SaveModelCallback(learn=learn)
rpcb = ReduceLROnPlateauCallback(learn=learn, patience=3)
csvcb = CSVLogger(learn=learn)
callbacks = [escb, smcb, rpcb, csvcb]


learn.fit(epochs=4, lr=1e-5, callbacks=callbacks)

epoch,train_loss,valid_loss,accuracy,time
0,6.690278,6.550413,0.262723,00:09
1,6.686635,6.532144,0.262723,00:09
2,6.679404,6.511220,0.262723,00:09
3,6.674406,6.489294,0.262723,00:09


Better model found at epoch 0 with valid_loss value: 6.550412654876709.
Better model found at epoch 1 with valid_loss value: 6.532143592834473.
Better model found at epoch 2 with valid_loss value: 6.5112199783325195.
Better model found at epoch 3 with valid_loss value: 6.489294052124023.


In [25]:
learn.export()

In [26]:
learn2 = load_learner(path)

In [30]:
! cat lang_model_onecycle/history.csv

epoch,train_loss,valid_loss,accuracy,time
0,3.726073,3.523209,0.375969,15:40:15
